# Project 2: Market-basket analysis - IMDB dataset

Project for the course of Algorithms for Massive Dataset <br> Nicolas Facchinetti 961648 <br> Antonio Belotti xxxxxx

# Download the dataset from Kaggle

First install the Python module of Kaggle to download the dataset from its datacenter

In [9]:
!pip install kaggle

Then load kaggle.json, a file containing your API credentials to be able to use the services offered by Kaggle

In [10]:
from google.colab import files

uploaded = files.upload()
  
# Move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


Now we can download the dataset

In [11]:
!kaggle datasets download 'ashirwadsangwan/imdb-dataset'

imdb-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


We now must unzip the compressed archive to use it. Once done we can also remove it.

In [12]:
!unzip imdb-dataset.zip && rm imdb-dataset.zip

Archive:  imdb-dataset.zip
  inflating: name.basics.tsv.gz      
  inflating: name.basics.tsv/name.basics.tsv  
  inflating: title.akas.tsv.gz       
  inflating: title.akas.tsv/title.akas.tsv  
  inflating: title.basics.tsv.gz     
  inflating: title.basics.tsv/title.basics.tsv  
  inflating: title.principals.tsv.gz  
  inflating: title.principals.tsv/title.principals.tsv  
  inflating: title.ratings.tsv.gz    
  inflating: title.ratings.tsv/title.ratings.tsv  


Lets try to load some data in a Pandas Dataframe

In [28]:
!cd name.basics.tsv && ls

name.basics.tsv


In [24]:
import pandas as pd 

# load and skip the bad lines only for testing the dataset
rca_df = pd.read_csv("name.basics.tsv.gz", compression='gzip', error_bad_lines=False)
rca_df.head()

b'Skipping line 297357: expected 6 fields, saw 7\n'
b'Skipping line 483132: expected 6 fields, saw 7\n'
b'Skipping line 585006: expected 6 fields, saw 7\nSkipping line 625459: expected 6 fields, saw 8\n'
b'Skipping line 765173: expected 6 fields, saw 9\n'
b'Skipping line 856380: expected 6 fields, saw 7\n'
b'Skipping line 2081388: expected 6 fields, saw 7\nSkipping line 2084405: expected 6 fields, saw 7\n'
b'Skipping line 2276284: expected 6 fields, saw 8\n'
b'Skipping line 2415383: expected 6 fields, saw 10\n'
b'Skipping line 2528729: expected 6 fields, saw 7\nSkipping line 2603985: expected 6 fields, saw 7\n'
b'Skipping line 2653917: expected 6 fields, saw 7\nSkipping line 2668019: expected 6 fields, saw 7\nSkipping line 2704944: expected 6 fields, saw 7\nSkipping line 2725330: expected 6 fields, saw 7\nSkipping line 2750343: expected 6 fields, saw 7\n'
b'Skipping line 2867876: expected 6 fields, saw 7\n'
b'Skipping line 3037435: expected 6 fields, saw 7\nSkipping line 3102104: expec

,,,,,nconst\tprimaryName\tbirthYear\tdeathYear\tprimaryProfession\tknownForTitles
nm0000001\tFred Astaire\t1899\t1987\tsoundtrack,actor,miscellaneous\ttt0050419,tt0053137,tt0043044,tt0072308
nm0000002\tLauren Bacall\t1924\t2014\tactress,soundtrack\ttt0117057,tt0037382,tt0071877,tt0038355,NaN
nm0000003\tBrigitte Bardot\t1934\t\N\tactress,soundtrack,producer\ttt0049189,tt0059956,tt0054452,tt0057345
nm0000004\tJohn Belushi\t1949\t1982\tactor,writer,soundtrack\ttt0078723,tt0080455,tt0072562,tt0077975
nm0000005\tIngmar Bergman\t1918\t2007\twriter,director,actor\ttt0050986,tt0083922,tt0069467,tt0050976


In [29]:
rca_df2 = pd.read_csv("name.basics.tsv/name.basics.tsv", error_bad_lines=False)
rca_df2.head()

b'Skipping line 297359: expected 6 fields, saw 7\nSkipping line 315476: expected 6 fields, saw 7\n'
b'Skipping line 483135: expected 6 fields, saw 7\n'
b'Skipping line 585009: expected 6 fields, saw 7\nSkipping line 625462: expected 6 fields, saw 8\n'
b'Skipping line 765176: expected 6 fields, saw 9\n'
b'Skipping line 856384: expected 6 fields, saw 7\n'
b'Skipping line 1311080: expected 6 fields, saw 8\n'
b'Skipping line 2076918: expected 6 fields, saw 7\nSkipping line 2079935: expected 6 fields, saw 7\n'
b'Skipping line 2271816: expected 6 fields, saw 8\n'
b'Skipping line 2410917: expected 6 fields, saw 10\n'
b'Skipping line 2524267: expected 6 fields, saw 7\nSkipping line 2599529: expected 6 fields, saw 7\n'
b'Skipping line 2649461: expected 6 fields, saw 7\nSkipping line 2663563: expected 6 fields, saw 7\nSkipping line 2700490: expected 6 fields, saw 7\nSkipping line 2720877: expected 6 fields, saw 7\nSkipping line 2745891: expected 6 fields, saw 7\n'
b'Skipping line 2863427: expect

,,,,,nconst\tprimaryName\tbirthYear\tdeathYear\tprimaryProfession\tknownForTitles
nm0000001\tFred Astaire\t1899\t1987\tsoundtrack,actor,miscellaneous\ttt0050419,tt0053137,tt0072308,tt0043044
nm0000002\tLauren Bacall\t1924\t2014\tactress,soundtrack\ttt0071877,tt0117057,tt0038355,tt0037382,NaN
nm0000003\tBrigitte Bardot\t1934\t\N\tactress,soundtrack,producer\ttt0054452,tt0049189,tt0059956,tt0057345
nm0000004\tJohn Belushi\t1949\t1982\tactor,writer,soundtrack\ttt0077975,tt0072562,tt0080455,tt0078723
nm0000005\tIngmar Bergman\t1918\t2007\twriter,director,actor\ttt0069467,tt0050976,tt0083922,tt0050986
